Run cells bellow to download and preprocess the **MovieLens20M** dataset.

Estimated time is around 45 minutes.

In [2]:
!wget https://download.wetransfer.com/eugv/9a05107f888d4194f358b85e283b8797c0af41b9e7bacf4a8473d4d4aefd/1bd4bcc32dfdbffd6a7a8038b69cb41e6c731581/wetransfer_ml-20m-zip_2023-04-24_1203.zip?token=eyJhbGciOiJIUzI1NiJ9.eyJpYXQiOjE2ODIzMzgzODMsImV4cCI6MTY4MjMzODk4MywidW5pcXVlIjoiNzk0NzhlZTBhYzU2Yjc3ODdjN2UwMTE4NjZjZmZiZmEyMDIzMDQyNDEyMDI0NiIsImZpbGVuYW1lIjoid2V0cmFuc2Zlcl9tbC0yMG0temlwXzIwMjMtMDQtMjRfMTIwMy56aXAiLCJ3YXliaWxsX3VybCI6Imh0dHA6Ly9zdG9ybS5wcm9kdWN0LnN2Yy5jbHVzdGVyLmxvY2FsOjkyOTIvYXBpL3dheWJpbGxzLzlhMDUxMDdmODg4ZDQxOTRmMzU4Yjg1ZTI4M2I4Nzk3YzBhZjQxYjllN2JhY2Y0YTg0NzNkNGQ0YWVmZCIsImZpbmdlcnByaW50IjoiMWJkNGJjYzMyZGZkYmZmZDZhN2E4MDM4YjY5Y2I0MWU2YzczMTU4MSIsImNhbGxiYWNrIjoie1wiZm9ybWRhdGFcIjp7XCJhY3Rpb25cIjpcImh0dHA6Ly9mcm9udGVuZC5zZXJ2aWNlLmV1LXdlc3QtMS53ZXRyYW5zZmVyLm5ldC93ZWJob29rcy9iYWNrZW5kXCJ9LFwiZm9ybVwiOntcInRyYW5zZmVyX2lkXCI6XCI3OTQ3OGVlMGFjNTZiNzc4N2M3ZTAxMTg2NmNmZmJmYTIwMjMwNDI0MTIwMjQ2XCIsXCJkb3dubG9hZF9pZFwiOjE4NDE5NTQ2OTM4LFwicmVjaXBpZW50X2lkXCI6XCI0NGExNTIyZmY0N2E0OTEwNmU0ZjAxY2UxZDZlYzRhOTIwMjMwNDI0MTIwMzE4XCJ9fSJ9.ne5EQxNN5TEtX8XBOuk8DJeyTz4yj8PJM_zykmjK99c&cf=y

In [9]:
#!unzip ml-20m.zip

In [13]:
import pandas as pd

All Items

In [14]:
m = pd.read_csv('ml-20m/movies.csv')
m = m[m.movieId.notnull()].reindex()
m['itemid']=m.movieId.apply(lambda x: str(int(x)))
m['product_name'] = m['title']
items = m[['itemid','product_name','genres']]
#items.to_json('items.json')
items.head()

,itemid,product_name,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Purchases and grouped purchases.

In [15]:
interactions = pd.read_csv('ml-20m/ratings.csv')
interactions = interactions[interactions.rating>=4.]
interactions = interactions.sort_values(['userId','timestamp'])
interactions['itemid'] = interactions['movieId'].apply(str)
interactions['userid'] = interactions['userId'].apply(str)
interactions['amount'] = 1
interactions['date'] = interactions['timestamp']
interactions[['itemid','userid','amount','date']]
#interactions.to_json("purchases.json")
interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
#iii.to_json('purchases_txt.json')
iii.head()

,userId,itemids
23,1,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
176,2,"62,1210,1356,1974,2951,110,589,1259,2948,3513,..."
257,3,"589,1721,2118,2710,2722,2448,2505,2694,858,122..."
424,4,"10,356,454,480,589,377,586,350,368,370,594,520..."
455,5,"62,141,736,780,671,832,1042,1393,150,590,380,4..."


Keep only users with 5 or more interactions.

Purchases.

In [16]:
purchases=pd.read_json('purchases.json')
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
purchases_item_counts = purchases[['userid','itemid']]
purchases_user_counts = purchases[['userid','itemid']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
pu5=purchases_user_count[purchases_user_count.nr_of_purchases>=5]
purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
#purchases_pu5.to_json('purchases_pu5.json')
purchases_pu5

,userId,movieId,rating,timestamp,itemid,userid,amount,date
23,1,1079,4.0,2004-09-10 03:07:45,1079,1,1,2004-09-10 03:07:45
96,1,2959,4.0,2004-09-10 03:08:18,2959,1,1,2004-09-10 03:08:18
111,1,3996,4.0,2004-09-10 03:08:47,3996,1,1,2004-09-10 03:08:47
6,1,151,4.0,2004-09-10 03:08:54,151,1,1,2004-09-10 03:08:54
58,1,1374,4.0,2004-09-10 03:09:06,1374,1,1,2004-09-10 03:09:06
...,...,...,...,...,...,...,...,...
19999942,138493,823,4.5,2009-12-07 18:13:09,823,138493,1,2009-12-07 18:13:09
20000257,138493,68319,4.5,2009-12-07 18:15:20,68319,138493,1,2009-12-07 18:15:20
20000213,138493,40819,4.5,2009-12-07 18:15:26,40819,138493,1,2009-12-07 18:15:26
20000200,138493,33794,4.0,2009-12-07 18:16:51,33794,138493,1,2009-12-07 18:16:51


Grouped purchases.

In [17]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userId','itemids']].drop_duplicates()
iii['userid']=iii['userId'].apply(str)
iii = iii[['userid','itemids']]
#iii.to_json('purchases_txt_pu5.json')
iii

/tmp/ipykernel_63201/3483715097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))


,userid,itemids
23,1,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
176,2,"62,1210,1356,1974,2951,110,589,1259,2948,3513,..."
257,3,"589,1721,2118,2710,2722,2448,2505,2694,858,122..."
424,4,"10,356,454,480,589,377,586,350,368,370,594,520..."
455,5,"62,141,736,780,671,832,1042,1393,150,590,380,4..."
...,...,...
19999614,138489,"1275,1747,3671,1061,2021,29,318,858,50,2019,91..."
19999644,138490,"111,3217,3654,3897,1535,2589,199,277,1735,1280..."
19999790,138491,"1093,4128,3186,2857,6874"
19999841,138492,"2115,908,58,2599,1500,1358,1288,1732,1252,1234..."


Users.

In [10]:
#iii['userid'].to_frame().to_json('users_pu5.json')
iii['userid'].to_frame().head()

,userid
23,1
176,2
257,3
424,4
455,5


Items

In [11]:
#items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")
items[items.itemid.isin(purchases_item_count_pu5.itemid)].head()

,itemid,product_name,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Items sorted by number of interactions

In [12]:
#purchases_item_count_pu5.to_json("items_sorted_pu5.json")
purchases_item_count_pu5.head()

,itemid,nr_of_purchases
6620,318,55705
6186,296,52265
12160,593,50061
7545,356,47302
4894,260,42578


Users sorted by number of interactions

In [ ]:
#pu5.to_json("users_sorted_pu5.json")
pu5.head()

,userid,nr_of_purchases
28626,125794,3177
118782,82418,2655
120595,8405,2503
20206,118205,2377
35395,131894,2349
...,...,...
4572,104116,5
28224,125431,5
11086,109989,5
67106,3584,5


Create train, val and test split.

In [19]:
users = pd.read_json('users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:10000]
val_users = shuffled_users.iloc[10000:20000]
train_users = shuffled_users.iloc[20000:]

#test_users.to_json("test_users.json")
print(test_users.head())
#val_users.to_json("val_users.json")
print(val_users.head())
#train_users.to_json("train_users.json") 
print(train_users.head())

len(train_users),len(val_users),len(test_users)

          userid
9483226    65567
18812883  130320
1125581     7677
5262991    36081
17249182  119275
          userid
15163263  104804
13123091   90664
11207540   77425
9316001    64428
13769119   95146
          userid
9141029    63177
17703308  122423
5668357    39001
1562218    10557
2448694    16563


(116677, 10000, 10000)

In [24]:
test_users.sort_index()

,userid
1192,11
1511,13
1770,14
5057,50
8149,73
...,...
19989248,138410
19991285,138422
19991592,138426
19994302,138449


List of generated json files with preprocessed data.

In [20]:
!ls *.json

VASP_ML20_1_loss.json	  purchases.json	  train_users.json
VASP_ML20_1_metrics.json  purchases_pu5.json	  users_pu5.json
items.json		  purchases_txt.json	  users_sorted_pu5.json
items_pu5.json		  purchases_txt_pu5.json  val_users.json
items_sorted_pu5.json	  test_users.json
